In [1]:
import sys 
sys.path.append('../')

from einops import rearrange
import torch 
from utils import mlgrid2d
from dataset import load_dataset_1d
from tqdm import trange
import argparse
from model import MLP, Rational, NMLK
import torch.nn as nn
from dataset import reference_test
from mlmm import *

In [2]:
# class NMLK(torch.nn.Module):
#     def __init__(self, layers, l=13, k=3, m=7):
#         super(NMLK, self).__init__()
#         self.k = k
#         self.m = m
#         self.n = 2**l + 1
#         model_dict = {}
#         model_dict['KHH'] = MLP(layers, Rational)
#         for i in range(k):
#             model_dict[f'Khh_band_even_{i+1}'] = MLP(layers, Rational)
#             model_dict[f'Khh_band_odd_{i+1}'] = MLP(layers, Rational)

#         self.models = nn.ModuleDict(model_dict)

#     def ml_grids(self, x, x_nbrs):
#         assert len(x) == self.n

#         # boundary coords on finest level
#         bd_coords = torch.cartesian_prod(x[[0,-1]], x)

#         # local coords on each level        
#         local_coords = torch.cartesian_prod(x, x_nbrs)
#         local_coords = rearrange(local_coords, '(n m) c -> 1 c n m', n = self.n, m=2*self.m+1, c=2)
#         local_coords_lst = []
#         for i in range(self.k):
#             local_coords_even = rearrange(local_coords[0,:,::2,::2], 'c m n -> m n c')
#             local_coords_odd = rearrange(local_coords[0,:,1::2], 'c m n -> m n c')
#             local_coords_lst.append([local_coords_even, local_coords_odd])
#             local_coords = injection1d_rows(local_coords)
#         local_coords_lst = local_coords_lst[::-1]

#         # full coords on coarest level
#         xH = local_coords[0,0,:,0]
#         gHx, gHy = torch.meshgrid(xH, xH, indexing='ij')
#         gHH = rearrange([gHx, gHy], 'b m n -> m n b')

#         return gHH, bd_coords, local_coords_lst

#     def ml_kernel(self, gHH, bd_coords, local_coords_lst):
#         # approximate KHH by MLP
#         KHH = self.models['KHH'](gHH)
#         KHH = rearrange(KHH, 'm n c -> 1 c m n')

#         # approximate Khh_bd by MLP
#         Khh_bd = self.models['KHH'](bd_coords)
#         Khh_bd = [Khh_bd[:self.n], Khh_bd[self.n:]]
#         Khh_bd = [rearrange(K, 'm n -> 1 1 n m') for K in Khh_bd]

#         # approximate Khh_banddiff_lst by MLP
#         Khh_banddiff_lst = []
#         for i in range(self.k):
#             Khh_banddiff_even = self.models[f'Khh_band_even_{i+1}'](local_coords_lst[i][0])
#             Khh_banddiff_odd = self.models[f'Khh_band_odd_{i+1}'](local_coords_lst[i][1])

#             Khh_banddiff_even = rearrange(Khh_banddiff_even, 'm n c -> 1 c m n')
#             Khh_banddiff_odd = rearrange(Khh_banddiff_odd, 'm n c -> 1 c m n')

#             Khh_banddiff_lst.append([Khh_banddiff_even, Khh_banddiff_odd])
        
#         return KHH, Khh_bd, Khh_banddiff_lst
    
#     def prepare_u_and_wb(self, uh, idx_j_lst, Khh_bd, h):
#         assert len(idx_j_lst) == k
#         uh_band_lst = []
#         lb_lst = []
#         rb_lst = []
#         for i in range(self.k):
#             n = uh.shape[-1]

#             # prepare uh_band
#             idx_mask = (idx_j_lst[i] >= 0) & (idx_j_lst[i] <= n-1)
#             uh_band = uh[:,:,idx_j_lst[i]] * idx_mask
#             uh_band_even = uh_band[:,:,::2]
#             uh_band_odd = uh_band[:,:,1::2]
#             uh_band_lst.append([uh_band_even[:,:,:,::2], uh_band_odd])
            
#             # calculate w boundaries
#             w_lb = multi_summation(Khh_bd[0], uh, h)
#             w_rb = multi_summation(Khh_bd[1], uh, h)
#             lb_lst.append(w_lb)
#             rb_lst.append(w_rb)

#             # coarsen Khh boundaries
#             Khh_bd[0] = injection1d_cols(Khh_bd[0])
#             Khh_bd[1] = injection1d_cols(Khh_bd[1])

#             # coarsen uh
#             uh = restrict1d(uh, order=order)
#             # coarsen h      
#             h = h*2

#         boundary_lst = [lb_lst[::-1], rb_lst[::-1]]
#         uh_band_lst = uh_band_lst[::-1]

#         return boundary_lst, uh_band_lst, uh, h

#     def forward(self, x, x_nbrs, uh, idx_j_lst, h):
#         # normalize grid to [0, 1]
#         x = (x - x[0])/(x[-1] - x[0])

#         # coarsest level full grids : gHH
#         # finest level boundary grids : bd_coords
#         # local coords on each level grids : local_coords_lst
#         gHH, bd_coords, local_coords_lst = self.ml_grids(x, x_nbrs)

#         # Prepare KHH, Khh_bd, Khh_banddiff_lst
#         Khh, Khh_bd, Khh_banddiff_lst = self.ml_kernel(gHH, bd_coords, local_coords_lst) # here Khh is KHH

#         # Prepare u_band and w_bd
#         boundary_lst, uh_band_lst, uh, h = self.prepare_u_and_wb(uh, idx_j_lst, Khh_bd, h) # here uh is uH

#         # coarsest approximation
#         wh = multi_summation(Khh, uh, h)    
#         wh_ml = [wh]

#         # multi-level correction
#         for i in range(k):
#             h = h/2        
#             wh_even_corr = (Khh_banddiff_lst[i][0]*uh_band_lst[i][0]).sum(axis=-1)*h
#             wh += wh_even_corr
#             wh[:,:,[0]] = boundary_lst[0][i]
#             wh[:,:,[-1]] = boundary_lst[1][i]
#             wh = interp1d(wh, order=order)
#             wh[:,:,1::2] = wh[:,:,1::2] + (Khh_banddiff_lst[i][1] * uh_band_lst[i][1]).sum(axis=-1)*h
#             wh_ml.append(wh)

#         return wh_ml, Khh, Khh_banddiff_lst

In [3]:
# data_root = '/workdir/pde_data/green_learning/data1d_8193/'
# task_nm = 'cosine'
# train_loader, test_loader, Khh, w_hom, xh, grid_pts, h = load_dataset_1d(
#     task_nm, data_root, bsz=64)

In [4]:
l = 13
k = 5
m = 15
order = 2
uh, Khh, h, wh_numeric, wh_analytic, xh = reference_test(l, 'cosine')
wh_singular_ml, KHH, Khh_banddiff_lst, boundary_lst, idx_j_lst = SingularSmoothKernelMLMM_local(
    uh, Khh, h, k, order, m)

In [7]:
nmlk = NMLK([2,50,50,1], k=k, m=m)

In [8]:
x = xh[0,0]
x_nbrs = (torch.arange(-m, m+1)/m +1)/2

In [9]:
wh_ml, nKhh, nKhh_banddiff_lst = nmlk(x, x_nbrs, uh, idx_j_lst, h)